**<h1>Hello,     
Welcome to the House Price Prediction Challenge, here we will design the algorithm to accurately predict house prices in India.<h1>**

<h2>1. Here we are importing all necessary modules.<h2>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline            

<h2>2. Importing datasets<h2>

In [ ]:
train = pd.read_csv('/kaggle/input/house-price-prediction-challenge/train.csv')
train.head()

In [ ]:
test = pd.read_csv('/kaggle/input/house-price-prediction-challenge/test.csv')
test.head()

<h2>3. Here we have 2 datasets, train and test data. As we know that train dataset is use to build the model and test dataset is use to test the performace of that model. So we will use train data here.<h2>

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
train.describe()

In [ ]:
pd.isna(train).sum()

In [ ]:
train.info()

In [ ]:
train['POSTED_BY'].value_counts()

In [ ]:
new = train['ADDRESS'].str.split(',', n=1,expand=True)

In [ ]:
train['ADDRESS'] = new[1]

In [ ]:
train.head()

In [ ]:
train['ADDRESS'].value_counts()

## 6. EDA

In [ ]:
i=1
plt.figure(figsize=(20,15))
for col in ['POSTED_BY', 'UNDER_CONSTRUCTION', 'RERA', 'BHK_NO.', 'BHK_OR_RK','READY_TO_MOVE', 'RESALE']:
    plt.subplot(4,2,i)
    sns.countplot(train[col])
    i+=1

In [ ]:
## the ready to move and under construction both columns are are not corelated with each other. we will drop one of the column.
train = train.drop(['READY_TO_MOVE'],axis=1)
train.head()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(train.corr(), annot=True, cmap='coolwarm')

In [ ]:
sns.scatterplot(x='LATITUDE', y='LONGITUDE', hue='TARGET(PRICE_IN_LACS)', data=train)

<h3>outliers are present in Longitude and lattitude columns.<h3>

## 7. Outliers handeling

In [ ]:
sns.boxplot(train['SQUARE_FT'])

In [ ]:
sns.boxplot(train['LONGITUDE'])

In [ ]:
sns.boxplot(train['LATITUDE'])

In [ ]:
## from above plots, all 3 columns have outliers. we wil use std normal distribution method here.
upper_boundry = train['LATITUDE'].mean() + (3 * train['LATITUDE'].std())
lower_boundry = train['LATITUDE'].mean() - (3 * train['LATITUDE'].std())
print(lower_boundry)
print(upper_boundry)
print(train['LATITUDE'].mean())

In [ ]:
train.drop(train[train['LATITUDE'] < 45.164455227196406].index, inplace=True)
train.drop(train[train['LATITUDE'] > 108.51093527034354].index, inplace=True)

In [ ]:
upper_boundry = train['LONGITUDE'].mean() + (3 * train['LONGITUDE'].std())
lower_boundry = train['LONGITUDE'].mean() - (3 * train['LONGITUDE'].std())
print(lower_boundry)
print(upper_boundry)
print(train['LONGITUDE'].mean())

In [ ]:
train.drop(train[train['LONGITUDE'] < 3.51830316623235].index, inplace=True)
train.drop(train[train['LONGITUDE'] > 38.78376796993674].index, inplace=True)

In [ ]:
train.shape

In [ ]:
import numpy as np
num_features= ['SQUARE_FT','LONGITUDE','LATITUDE']
for feature in num_features:
    train[feature] = np.log(train[feature])

In [ ]:
upper_boundry = train['SQUARE_FT'].mean() + (3 * train['SQUARE_FT'].std())
lower_boundry = train['SQUARE_FT'].mean() - (3 * train['SQUARE_FT'].std())
print(lower_boundry)
print(upper_boundry)
print(train['SQUARE_FT'].mean())

In [ ]:
train.drop(train[train['SQUARE_FT'] < 5.30187195004729].index, inplace=True)
train.drop(train[train['SQUARE_FT'] > 8.870739603915984].index, inplace=True)

In [ ]:
train.shape

In [ ]:
train.isnull().sum()

In [ ]:
X = train.drop(['TARGET(PRICE_IN_LACS)','BHK_OR_RK'],axis=1)
y = train['TARGET(PRICE_IN_LACS)']

In [ ]:
X.head()

In [ ]:
## Spliting the data 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=5)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt
from sklearn import set_config
set_config(display='diagram')
import warnings 
warnings.filterwarnings('ignore')

In [ ]:
numeric_features = ['SQUARE_FT','LONGITUDE','LATITUDE']
numeric_transformer = MinMaxScaler()

preprocessor = ColumnTransformer(transformers=[('numeric',numeric_transformer,numeric_features)])

In [ ]:
categorical_features = ['POSTED_BY','UNDER_CONSTRUCTION','RERA','BHK_NO.','RESALE','ADDRESS']
categorical_transformer = OrdinalEncoder(handle_unknown='ignore')
preprocessor_ = ColumnTransformer(transformers=[('numeric',numeric_transformer,numeric_features),
                                              ('categorical',categorical_transformer,categorical_features)])

In [ ]:
preprocessor_

In [ ]:
## we will check the accuracy for various model.
pipeline_lr = Pipeline([('preprocessor',preprocessor_),('lr_regressor',LinearRegression())])

In [ ]:
lr_model = pipeline_lr.fit(X_train,y_train)
y_pred_lr = lr_model.predict(X_test)
print('SME:',sqrt(mean_squared_error(y_test,y_pred_lr)))
print(r2_score(y_test,y_pred_lr))

In [ ]:
pipeline_rf = Pipeline([('preprocessor',preprocessor_),('rf_regressor',RandomForestRegressor())])

In [ ]:
rf_model = pipeline_rf.fit(X_train,y_train)
y_pred_rf = rf_model.predict(X_test)
print('SME:',sqrt(mean_squared_error(y_test,y_pred_rf)))
print(r2_score(y_test,y_pred_rf))

In [ ]:
pipeline_dt = Pipeline([('preprocessor',preprocessor_),('dt_regressor',DecisionTreeRegressor())])

In [ ]:
dt_model = pipeline_dt.fit(X_train,y_train)
y_pred_dt = dt_model.predict(X_test)
print('SME:',sqrt(mean_squared_error(y_test,y_pred_dt)))
print(r2_score(y_test,y_pred_dt))

In [ ]:
pipeline_gb = Pipeline([('preprocessor',preprocessor_),('gb_regressor',GradientBoostingRegressor())])

In [ ]:
gb_model = pipeline_gb.fit(X_train,y_train)
y_pred_gb = gb_model.predict(X_test)
print('SME:',sqrt(mean_squared_error(y_test,y_pred_gb)))
print(r2_score(y_test,y_pred_gb))

#### The random forest regressor perform better. we will use hyperparameter tuning technique to improve the performance 

In [ ]:
X_train.head()

In [ ]:
preprocessor_.fit(X_train)

In [ ]:
X_train = pd.DataFrame(preprocessor_.transform(X_train))
X_train.head()

In [ ]:
X_test = pd.DataFrame(preprocessor_.transform(X_test))
X_test.head()

In [ ]:
import pickle
filename = 'finalized_model.pkl'
pickle.dump(rf_model, open(filename, 'wb'))

In [ ]:
## we need to rearange the test dataset columns as per train data 
new = test['ADDRESS'].str.split(',',n=1,expand=True)
test['ADDRESS'] = new[1]
test.head()

In [ ]:
test = test.drop(['BHK_OR_RK','READY_TO_MOVE'],axis=1)
test.head()

In [ ]:
## Now we will check the accuracy for test dataset. 
prediction_test = rf_model.predict(test)
prediction_test